In [1]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
from aruco import detect_aruco, draw_aruco, select_aruco_poses, PoseSelectors, \
    select_aruco_markers
from segmentation import segment_and_draw_boxes_by_aruco, segment_scene
from calibrate_table import calibrate_table
from utils import show, stream_table_frame, stream_segmented_scene, \
    stream_aruco_detected_on_boxes
from aruco_detection_configs import aruco_dict, aruco_detection_params, retry_rejected_params

In [2]:
calib = np.load('phitz/calib.npz')
K = calib['K']
D = calib['D']

In [3]:
table_aruco_size = 0.132
box_aruco_size = 0.0172
box_size = 0.03

In [ ]:
# calibrate table
image_file = "phitz/table_markers/1.jpg"
image = cv2.imread(image_file)
camera2table, _ = calibrate_table(image, K, D, table_aruco_size)
if camera2table is None:
    print("Could not detect table")
else:
    rvec, _ = cv2.Rodrigues(camera2table[0:3, 0:3])
    tvec = camera2table[0:3, 3]
    draw = image.copy()
    cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.1)
    show(draw)

In [9]:
# detect table aruco
image_file = "phitz/table_markers/1.jpg"
image = cv2.imread(image_file)
arucos = detect_aruco(image, K=K, D=D, aruco_sizes=table_aruco_size, use_generic=True,
    aruco_dict=aruco_dict, params=aruco_detection_params)
arucos = select_aruco_poses(arucos, PoseSelectors.Z_axis_up)
arucos = select_aruco_markers(arucos, lambda id: id < 4)
draw = image.copy()
draw_aruco(draw, arucos, draw_rejected_only=False, draw_ids=False, K=K, D=D)
show(draw)

In [ ]:
# detect box aruco
folder = "phitz/test_aruco/"
image_files = sorted(os.listdir(folder))
bad_image_files = list()
for image_file in image_files:
    image = cv2.imread(folder + image_file)
    arucos = detect_aruco(image, K=K, D=D, aruco_sizes=box_aruco_size, use_generic=True,
        retry_rejected=True, retry_rejected_params=retry_rejected_params,
        aruco_dict=aruco_dict, params=aruco_detection_params)
    arucos = select_aruco_poses(arucos, PoseSelectors.Z_axis_up)
    arucos = select_aruco_markers(arucos, lambda id: id >= 4)
    draw = image.copy()
    draw_aruco(draw, arucos, draw_rejected_only=False, draw_ids=False, K=K, D=D)
    # show(draw)
    # cv2.imwrite("phitz/test_aruco_vis/" + image_file, draw)
    if arucos.n != 2:
        print(f"{image_file}: {arucos.n}")
        bad_image_files.append(image_file)
print(len(bad_image_files))
print(bad_image_files)

In [ ]:
# segment scene
image_file = "phitz/test/0000.jpg"
image = cv2.imread(image_file)
segmentation, (num_red, num_blue) = segment_scene(image)
print(f"Segmented {num_red} red, {num_blue} blue")
show(segmentation)
# cv2.imwrite("segmentation.jpg", segmentation)

In [ ]:
# open camera
cam = cv2.VideoCapture(2)
camera = lambda: cam.read()[1]

In [ ]:
# stream table frame
save_folder = None
stream_table_frame(camera, K, D, table_aruco_size, save_folder=save_folder)

In [ ]:
# stream segmented scene
save_folder = None
stream_segmented_scene(camera, save_folder=save_folder)

In [ ]:
# stream aruco detected on boxes
save_folder = None
stream_aruco_detected_on_boxes(camera, K, D, box_aruco_size, save_folder=save_folder)

In [ ]:
# close camera
cam.release()